In [1]:
print("Model of Machine Learning based on K nearest neightbour")

Model of Machine Learning based on K nearest neightbour


In [1]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display
import pandas as pd
import pymysql
from datetime import datetime, timedelta 
import math

This model is based on the K Nearest Neighbour algorithm. The input elements are weather conditions, wind speed, time, and day of the week. Usually, these elements can most affect the rider's willingness to ride. I first calculate the distance between the forecast and the historical data (the difference between each element is substituted into the distance formula). Then select the top five of the shortest distance, that is, k is equal to 5. Then use the weighted average to calculate the average. This average value is the predicted value.<br>
The knn algorithm was chosen because the model does not require training while ensuring a certain accuracy.

In [2]:
def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums) - i - 1):  
            if nums[j][0] > nums[j + 1][0]:
                nums[j], nums[j + 1] = nums[j + 1], nums[j]

    return nums

In [11]:
def get_prediction_data(date,time):
    dic_w = {'Clear':1,'Clouds':2,'Mist':3,'Drizzle':4,'Fog':5,'Rain':6,'Snow':7}
    weekday = datetime.strptime(date,"%Y-%m-%d").weekday()
    #day = datetime.strptime(date,"%Y-%m-%d").day
    time = str(time) + ":00:00"
    dic = get_forecast_test(date + ' ' + time)
    con = dic_w[dic['condition']]
    dic['condition'] = con
    dic['time'] = date + ' ' + time
    dic['weekday'] = weekday
    return dic

def get_forecast(date):
    day = datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
    day_now = datetime.now()
    day = (day - day_now).total_seconds()/3600
    hour = int(day / 3)
    url = 'https://api.openweathermap.org/data/2.5/forecast?lat=53.36&lon=-6.25&appid=dbb2b9eb4f9424b9c2c168ad52c077d9'
    info = requests.get(url).json()
    wind_speed = info['list'][hour]['wind']['speed']
    condition = info['list'][hour]['weather'][0]['main']
    temp = info['list'][hour]['main']['temp']
    return {'wind':wind_speed,'condition':condition,'temp':temp}

def get_forecast_test(date):
    dic_weather = {}
    low = datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
    high = low + timedelta(hours = 8)
    df = pd.read_csv('old_weather.csv')
    df['datetime'] = df['datetime'].astype('datetime64')
    temp=df[(df['datetime']>=low) & (df['datetime']<=high)]
    dic_weather['condition'] = temp.iloc[[0],[3]].values[0][0]
    dic_weather['wind'] = temp.iloc[[0],[9]].values[0][0]
    dic_weather['temp'] = temp.iloc[[0],[6]].values[0][0]
    return dic_weather

In [38]:
def get_csv():
    low = '2020-03-04 09:39:43'
    high = '2020-03-24 09:47:28'
    sql = "SELECT * FROM dbike.weather where `datetime` between '"+ low +"' and '"+ high +"';"
    conn = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    df = pd.read_sql(sql, con=conn)
    df.to_csv("old_weather.csv",index=False)

In [39]:
def get_old_weather(low,df):
    dic_old_weather = {}
    dic_w = {'Clear':1,'Clouds':2,'Mist':3,'Drizzle':4,'Fog':5,'Rain':6,'Snow':7}
    #low = datetime.strptime(low,"%Y-%m-%d %H:%M:%S")
    high = low + timedelta(hours = 8)
    #low = '2020-03-04 15:20:02'
    #high = '2020-03-04 15:40:02'
    temp=df[(df['datetime']>=low) & (df['datetime']<=high)]
    #temp.head(100)
    if temp.empty:
        return dic_old_weather
    temp.iloc[[0],[3]].values[0][0]
    #print(data)
    dic_old_weather['condition'] = dic_w[temp.iloc[[0],[3]].values[0][0]]
    dic_old_weather['wind'] = temp.iloc[[0],[9]].values[0][0]
    dic_old_weather['temp'] = temp.iloc[[0],[6]].values[0][0]
    return dic_old_weather

def prediction(number,date,time):
    #get_csv()
    calender = date + ' ' + "00:00:00"
    db = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    cursor = db.cursor()
    sql = "SELECT * FROM dbike.daily_data where `number` = "+ str(number) + " and `last_update` between '2020-03-03 12:00:02' and '" + calender + "' group by last_update;"
    cursor.execute(sql)
    db.commit()
    data = cursor.fetchall()
    cursor.close()
    length = len(data)
    i = 0
    count = 0
    dic_pre = get_prediction_data(date,time)
    date_new = datetime.strptime(dic_pre['time'],"%Y-%m-%d %H:%M:%S")
    li = []
    #print(length)
    df = pd.read_csv('old_weather.csv')
    df['datetime'] = df['datetime'].astype('datetime64')
    while i < length:
        date_old = data[i][4]
        available_bike_stands = data[i][2]
        available_bikes = data[i][3]
        weekday_old = date_old.weekday()
        dic_old = get_old_weather(date_old,df)
        i = i + 1
        if count == 5:
            break
        if not dic_old:
            continue
        d_condition = float(dic_pre['condition'])-float(dic_old['condition'])
        d_wind = (float(dic_pre['wind']) - float(dic_old['wind']))*5
        if date_new.hour*10000+date_new.minute*100+date_new.second > date_old.hour*10000+date_old.minute*100+date_old.second:
            d_time = (date_new - date_old).seconds/3600
        else:
            d_time = (date_old - date_new).seconds/3600
        d_week = int(dic_pre['weekday'])-int(weekday_old)
        d_time = round(d_time,3)
        #print(d_condition)
        #print(d_wind)
        #print(d_time,end=" ")
        #print(d_week)
        distance = math.sqrt(d_wind**2 + d_condition**2 + d_time**2 + d_week**2)
        li.append([distance,available_bikes,available_bike_stands])
        #print(li)
    #print(len(li))
    bubble_sort(li)
    #print(li[0],li[1],li[2])
    weight = 1/li[0][0] + 1/li[1][0] + 1/li[2][0] + 1/li[3][0] + 1/li[4][0]
    prediction_bikes = (int(li[0][1])*(1/li[0][0]) + int(li[1][1])*(1/li[1][0]) + int(li[2][1])*(1/li[2][0]) + int(li[3][1])*(1/li[3][0]) + int(li[4][1])*(1/li[4][0]))/ weight
    prediction_stands = (int(li[0][2])*(1/li[0][0]) + int(li[1][2])*(1/li[1][0]) + int(li[2][2])*(1/li[2][0]) + int(li[3][2])*(1/li[3][0]) + int(li[4][2])*(1/li[4][0]))/weight
    dic = {}
    dic_w = {'1':'Clear','2':'Clouds','3':'Mist','4':'Drizzle','5':'Fog','6':'Rain','7':'Snow'}
    dic['prediction_bikes']=prediction_bikes
    dic['prediction_stands']=prediction_stands
    dic['condition'] = dic_w[str(dic_pre['condition'])]
    dic['wind'] = dic_pre['wind']
    dic['temp'] = dic_pre['temp']
    return dic

In [41]:
def actual_data(number,date,time):
    calender = date + ' ' + time + ":00:00"
    db = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    cursor = db.cursor()
    sql = "SELECT * FROM dbike.daily_data where `number` = "+ str(number) + " and `last_update` between '" + calender + "' and '2020-04-01 07:20:02' group by last_update;"
    cursor.execute(sql)
    db.commit()
    data = cursor.fetchall()
    cursor.close()
    available_bike_stands = data[0][2]
    available_bikes = data[0][3]
    print("Actual_bikes is",available_bikes,"Actual_stands is",available_bike_stands)

In [42]:
def test(station,times):
    i = 1
    day = 18
    while i <= times:
        date = '2020-03-' + str(day+i)
        print("-------------test ",i," start--------------")
        print(date)
        dic = prediction(station,date,'09')
        print("prediction_bikes is",int(dic['prediction_bikes']),"prediction_stands is",int(dic['prediction_stands']))
        actual_data(station,date,'09')
        print()
        print("-------------test ",i," end--------------")
        print()
        i = i + 1


In [43]:
test(32,5)

-------------test  1  start--------------
2020-03-19
prediction_bikes is 5 prediction_stands is 24
Actual_bikes is 3 Actual_stands is 27

-------------test  1  end--------------

-------------test  2  start--------------
2020-03-20
prediction_bikes is 5 prediction_stands is 24
Actual_bikes is 7 Actual_stands is 23

-------------test  2  end--------------

-------------test  3  start--------------
2020-03-21
prediction_bikes is 11 prediction_stands is 18
Actual_bikes is 14 Actual_stands is 16

-------------test  3  end--------------

-------------test  4  start--------------
2020-03-22
prediction_bikes is 8 prediction_stands is 21
Actual_bikes is 9 Actual_stands is 21

-------------test  4  end--------------

-------------test  5  start--------------
2020-03-23
prediction_bikes is 12 prediction_stands is 15
Actual_bikes is 4 Actual_stands is 26

-------------test  5  end--------------



In [44]:
test(40,5)

-------------test  1  start--------------
2020-03-19
prediction_bikes is 5 prediction_stands is 14
Actual_bikes is 8 Actual_stands is 13

-------------test  1  end--------------

-------------test  2  start--------------
2020-03-20
prediction_bikes is 9 prediction_stands is 11
Actual_bikes is 13 Actual_stands is 8

-------------test  2  end--------------

-------------test  3  start--------------
2020-03-21
prediction_bikes is 7 prediction_stands is 13
Actual_bikes is 0 Actual_stands is 21

-------------test  3  end--------------

-------------test  4  start--------------
2020-03-22
prediction_bikes is 7 prediction_stands is 13
Actual_bikes is 5 Actual_stands is 16

-------------test  4  end--------------

-------------test  5  start--------------
2020-03-23
prediction_bikes is 12 prediction_stands is 8
Actual_bikes is 18 Actual_stands is 3

-------------test  5  end--------------



In [46]:
test(6,5)

-------------test  1  start--------------
2020-03-19
prediction_bikes is 2 prediction_stands is 17
Actual_bikes is 6 Actual_stands is 14

-------------test  1  end--------------

-------------test  2  start--------------
2020-03-20
prediction_bikes is 10 prediction_stands is 9
Actual_bikes is 11 Actual_stands is 9

-------------test  2  end--------------

-------------test  3  start--------------
2020-03-21
prediction_bikes is 4 prediction_stands is 15
Actual_bikes is 0 Actual_stands is 20

-------------test  3  end--------------

-------------test  4  start--------------
2020-03-22
prediction_bikes is 1 prediction_stands is 18
Actual_bikes is 1 Actual_stands is 19

-------------test  4  end--------------

-------------test  5  start--------------
2020-03-23
prediction_bikes is 19 prediction_stands is 0
Actual_bikes is 7 Actual_stands is 13

-------------test  5  end--------------

